# 7. Attention机制
本节旨在介绍[Attention机制](https://zhuanlan.zhihu.com/p/46313756/)与两种典型的应用（Self-attention, Multi-head attention），及其实现方法。

## 7.1 Attention
以NLP领域为例，常规的encoding是无法体现对一个句子序列中不同语素的关注程度的，然而一个句子中不同部分具有不同含义，并在意义上具有不同的重要性。

Attention机制是一种能让模型对重要信息重点关注并充分吸收的技术，能够作用于任何序列模型中。其通过赋予序列中不同语素以不同权重，结合实际场景的优化目标（如情感分析将着重关注Like/Dislike这种语素），来实现对不同语素进行不同侧重的目标。

### 7.1.1 Attention机制流程
**下面以seq2seq模型为例，阐述attention最基本的流程：**

对于一个包含有n个单词的句子序列source $S=[w_1, w_2, \cdots, w_n]$
1. 应用某种方法将 $S$ 的每个单词 $w_i$ 编码为一个单独的向量 $v_i$；
<p align=center>
<img src="./fig/7-3.png" width=700>
</p>

2. decoding阶段，使用学习到的Attention权重 $a_i$ 对1中得到的所有向量做线性加权 $\sum_i a_iv_i$。
<p align=center>
<img src="./fig/7-4.png" width=700>
</p>

3. 在decoder进行下一个单词的预测时，使用2中得到的线性组合。
<p align=center>
<img src="./fig/7-5.png" width=700>
</p>


由此可以抽象出Attention实现的三要素，Query，Key，Value，其中Q与K用于计算线性权重，V用于加权

<p align=center>
<img src="./fig/7-1.png" width=700>
</p>

对于Q, K, V的例子理解：

<p align=center>
<img src="./fig/7-2.png" width=700>
</p>

### 7.1.2 Attention的核心-注意力权重计算
Attention机制的核心在于如何通过Query和Key计算注意力权重，下面总结常用的几个方法：

1. 多层感知机(Multi-Layer Perception, MLP)
$$ a(q,k) = w_2^T tanh(W_1 [q;k])$$
首先将向量$q$与$k$进行拼接，经过全连接$W_1$线性映射后，$tanh$激活，通过一个全连接$w_2$线性映射至一个值。

MLP方法训练成本高，对大规模数据较为有效。

2. Bilinear
$$ a(q,k) = q^TWk$$
通过一个权重矩阵$W$建立$q$与$k$之间的相关关系，简单直接，计算速度快。

3. Dot Product
$$ a(q,k) = q^Tk$$
直接建立$q$与$k$之间的相关关系（内积，相似度），要求二者维度相同。

4. Scaled-dot Product
对3的改进，由于q和k的维度增加，会使得最后得到的内积a可能也会变得很大，这使得后续归一化softmax的梯度会非常小，不利于模型训练。参考[为什么dot-product需要被scaled](https://blog.csdn.net/qq_37430422/article/details/105042303)
$$ a(q,k) = \frac{q^Tk}{\sqrt{|k|}}$$
通过k的模长对a的尺度进行scaled，避免梯度消失问题。

## 7.2 Self-attention
Self-attention是attention机制的一种应用，其中，attention完成了输入source和输出target之间的加权映射。而self-attention字如其名，通过使得source=target，自己对自己本身进行注意力机制计算，来捕获序列数据自身的相互依赖特性。

即，在一般任务的Encoder-Decoder框架中，输入Source和输出Target内容是不一样的，比如对于英-中机器翻译来说，Source是英文句子，Target是对应的翻译出的中文句子，Attention机制发生在Target的元素Query和Source中的所有元素之间。

而Self-attention的注意力机制，是在Source=Target的特殊情况下，内部元素之间的attention机制，其具体计算过程是一样的，只是计算对象发生了变化而已。

<p align=center>
<img src="./fig/7-6.png" width=700>
</p>

如上图所示，我们将句子做self-attention，可以看到source中的语素'its'的attention集中在target中的语素'Law'与'application'上，这种self-attention使我们能够捕获这个句子内部不同元素间的依赖关系。

很明显，引入self-attention后，序列数据中长距离的相互依赖性将更容易被捕获。对于RNN来说，依次序列计算难以捕获远距离的依赖性，但self-attention通过直接将序列数据中任意两个样本的联系通过一个计算步骤直接联系起来，极大地缩短了远距离依赖性的距离，有利于有效地利用这些远距离相互依赖的特征。

## 7.3 Multi-head attention
另一种有效的应用为multi-head attention,其主要思想为从**多视角**看待（Q, K, V）的attention映射关系，是attention的拓展版本。

Multi-head attention通过设计h种不同的权重矩阵对 $(W_i^Q, W_i^K, W_i^V)_{i=1}^h$ 对 $ (Q, K, V)$进行attention计算，得到h个不同的 ${head_i}_{i=1}^h$ ，而后concat起来做一个全连接 $W^o$得到最后的attention输出，如图所示：

<p align=center>
<img src="./fig/7-7.png" width=300>
</p>

$$head_i = attention(QW^Q, KW^K, VW^V)$$
$$output = multihead(Q, K, V) = [head_1, \cdots, head_h]W^o$$

关于全连接 $(W_i^Q, W_i^K, W_i^V)$ 的输出维度 $(d^q, d^k, d^v)$，通常小于 $ (Q, K, V)$ 的输入维度 $d$，因为multi-head的计算成本过高，维度的增加将大大增加算法计算量，一般采用：

$$d^q=d^k=d^v=d/h$$